# Plan de Refactorización Profesional

Este notebook describe, por área, qué mejorar y cómo llevar el proyecto de análisis de encuestas a un estándar profesional: más modular, robusto, mantenible y escalable.

## 1. Estructura y Modularidad

Diagnóstico:  
- Módulos dispersos (`file_loader.py` vs `src/data_loader.py`, `untitled5.py` no integrado).  

Recomendaciones:  
- Eliminar o migrar `untitled5.py` a funciones de `main.py` o a un nuevo módulo `src/pipeline.py`.  
- Unificar `file_loader.py` y `src/data_loader.py` en `src/data_loader.py`.  
- Adoptar un layout tipo:
  ```
  Proyecto_Estadistica/
  ├─ src/
  │  ├─ __init__.py
  │  ├─ data_loader.py
  │  ├─ data_processor.py
  │  ├─ visualizer.py
  │  └─ ai_analyzer.py
  ├─ utils/
  ├─ main.py
  └─ tests/
  ```
- Modularizar cada paso en funciones o clases (p.ej. `class DataLoader`, `class DataProcessor`).

## 2. Carga y Validación de Datos (data_loader.py)

Diagnóstico:  
- Depende de Tkinter, poca validación de esquema.  

Recomendaciones:  
- Reemplazar selección gráfica por parámetro CLI o env var:  
  ```python
  def load_data(path=None):
      if not path:
          path = os.getenv("DATA_PATH") or sys.argv[1]
      ...
  ```
- Usar `pydantic` o `pandera` para validar columnas y tipos al leer CSV/Excel.  
- Registrar cada paso con `logging` (`logger.info("Leyendo archivo %s", path)`).  
- Soportar iterables de datos (JSON, SQL) en el mismo módulo.

## 3. Procesamiento y Limpieza de Datos (data_processor.py)

Diagnóstico:  
- Limpieza básica (NaN, fechas, colas).  

Recomendaciones:  
- Usar pipelines de pandas o `sklearn.Pipeline` para encadenar transformaciones.  
- Añadir validación estadística (detección de outliers con IQR o Z-score).  
- Imputación avanzada (por agrupamiento o modelos simples).  
- Extraer features: edad del encuestado, antigüedad, NPS de PREGUNTA_2.  
- Normalizar/limpiar texto con `spaCy` o `nltk` (lemmatización, stopwords).

## 4. Análisis Estadístico y Visualizaciones (visualizer.py)

Diagnóstico:  
- Gráficos estáticos con matplotlib/seaborn.  

Recomendaciones:  
- Migrar a `plotly.express` para interactividad; empaquetar en un dashboard con `Dash` o `Streamlit`.  
- Centralizar estilos (tema, paleta) en utils.  
- Añadir análisis descriptivo avanzado: distribuciones por segmento, test de hipótesis (t-test, ANOVA).  
- Documentar cada función con `docstring` y ejemplos.

## 5. Integración con IA (ai_analyzer.py)

Diagnóstico:  
- Prompt muy largo, sin manejo de errores.  

Recomendaciones:  
- Definir plantillas de prompt con `jinja2` y secciones opcionales.  
- Implementar retries y backoff con `tenacity`.  
- Procesar respuestas con validación JSON o esquema (p.ej. extrayendo listados de insights).  
- Permitir al usuario elegir tipo de informe (resumen, temas, anomalías) vía config.

## 6. Configuración (config_utils.py y config.ini)

Diagnóstico:  
- Config plano, API keys en texto claro.  

Recomendaciones:  
- Cargar secrets desde variables de entorno (p.ej. `AIApiKey`).  
- Soportar múltiples entornos (dev/prod) con archivos `config.dev.ini`, `config.prod.ini`.  
- Usar `python-dotenv` para entornos locales.

## 7. Manejo de Errores y Logging

Diagnóstico:  
- Prints y exits puntuales.  

Recomendaciones:  
- Configurar `logging` con niveles y handlers (archivo, consola).  
- Definir excepciones propias (`class DataLoadError(Exception)`).  
- Capturar y loggear errores en cada etapa del pipeline sin abortar todo (o con retries).

## 8. Calidad de Código y Mantenibilidad

Diagnóstico:  
- Código sin typing, estilo mixto.  

Recomendaciones:  
- Adoptar PEP-8 y `flake8`, `black`.  
- Añadir `mypy` para tipado estático.  
- Mantener docstrings en formato Google o NumPy.  
- Dividir funciones >50 líneas.

## 9. Documentación (README.md)

Diagnóstico:  
- Mezcla de notas de campo y guía de usuario.  

Recomendaciones:  
- Reestructurar en secciones:  
  1. Descripción  
  2. Instalación y requisitos  
  3. Estructura del proyecto  
  4. Uso: CLI y ejemplos  
  5. Configuración  
  6. Desarrollo y testing  
  7. Contribuciones  
  8. Licencia

## 10. Testing

Diagnóstico:  
- Sin tests.  

Recomendaciones:  
- Escribir tests unitarios con `pytest` para cada módulo:  
  - Carga de datos (mock CSV).  
  - Limpieza de datos (DataFrame de prueba).  
  - Visualizaciones (funciones devuelven figuras).  
  - IA (mock de API).  
- Añadir CI (GitHub Actions) para ejecutar tests y linters.